# 모델링
## TF-IDF를 활용한 모델 구현

In [1]:
import pandas as pd

In [27]:
data_in_path = './data_in/'
train_clean_data = 'train_clean.csv'

train_data = pd.read_csv(data_in_path + train_clean_data, header=0)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

### TF-IDF 벡터화

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True, ngram_range=(1, 3), max_features=5000)

X = vectorizer.fit_transform(reviews)

NameError: name 'review' is not defined

### 학습과 검증 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split
import numpy as np

random_seed = 42
test_split = 0.2

y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=test_split, random_state=random_seed)
print(X_train.shape, y_train.shape)
print(X_eval.shape, y_eval.shape)

(20000, 5000) (20000,)
(5000, 5000) (5000,)


### 모델 선언 및 학습

In [14]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 검증 데이터로 성능 평가

In [15]:
print('Accuracy:', lgs.score(X_eval, y_eval))

Accuracy: 0.8598


### 데이터 제출

In [30]:
test_clean_data = 'test_clean.csv'

test_data = pd.read_csv(data_in_path + test_clean_data, header=0)
test_data.head()

,review,id
0,naturally film main themes mortality nostalgia...,12311_10
1,movie disaster within disaster film full great...,8348_2
2,movie kids saw tonight child loved one point k...,5828_4
3,afraid dark left impression several different ...,7186_2
4,accurate depiction small time mob life filmed ...,12128_7


In [21]:
testDataVecs = vectorizer.transform(test_data['review'])

In [22]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [31]:
import os
data_out_path = './data_out/'

if not os.path.exists(data_out_path):
    os.makedirs(data_out_path)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({
    'id': ids,
    'sentiment': test_predicted
})
answer_dataset.to_csv(data_out_path + 'lgs_tfidf_answer.csv', index=False)